In [1]:
import requests
import pprint
from bs4 import BeautifulSoup
import pandas as pd 


In [2]:
fields = ['transport','data','cloud']

In [3]:
# need to build list of cities
# iterate each cities with get requests
soups = []

# need a way to check pagination container for list length, thats how we know how long it is
for field in fields:
    URL = 'https://www.postgrad.com/search/chevening/?q='+ field + '&uk_location='
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    soup = (soup, field)
    soups.append(soup)


In [4]:
#use the initial html soup scraped to figure out how many pages each query has
pages_list = []

for soup,field in soups:
    page_container = soup.find('div', class_='pagination-container')
    pages = page_container.find_all('li')

    #number of of <li> indicates number of pages. -1 because 1 <li> represents the active page
    if len(pages) != 1:
        pages_list.append(len(pages)-1)
    else:
        pages_list.append(1)


In [5]:
field_page_dict = dict(zip(fields, pages_list))
new_soups = []
for field,pages in field_page_dict.items():
    for page in range(1,pages+1):
        URL = 'https://www.postgrad.com/search/chevening/?q='+ field + '&uk_location=&page=' + str(page)
        print(URL)
        page = requests.get(URL)
        new_soup = BeautifulSoup(page.content, 'html.parser')
        temp_tuple = (new_soup , field)
        new_soups.append(temp_tuple)

https://www.postgrad.com/search/chevening/?q=transport&uk_location=&page=1
https://www.postgrad.com/search/chevening/?q=transport&uk_location=&page=2
https://www.postgrad.com/search/chevening/?q=data&uk_location=&page=1
https://www.postgrad.com/search/chevening/?q=data&uk_location=&page=2
https://www.postgrad.com/search/chevening/?q=data&uk_location=&page=3
https://www.postgrad.com/search/chevening/?q=data&uk_location=&page=4
https://www.postgrad.com/search/chevening/?q=data&uk_location=&page=5
https://www.postgrad.com/search/chevening/?q=data&uk_location=&page=6
https://www.postgrad.com/search/chevening/?q=data&uk_location=&page=7
https://www.postgrad.com/search/chevening/?q=data&uk_location=&page=8
https://www.postgrad.com/search/chevening/?q=cloud&uk_location=&page=1


In [6]:
sections_list =[]

for new_soup , field in new_soups:
    sections = new_soup.find_all('section', class_='search_dept')
    temp_tup = (sections,field)
    sections_list.append(temp_tup)


In [7]:
def dataFramer(sections):
    unis = []
    courses = []
    chevenings = []
    links = []
    fields = []
    field = sections[1]
    for section in sections[0]:
        uni = section.find('div', class_='search_dept-head')
        #Try to find unis that is are chevening
        try :
            chevening = uni.find('a', class_='tag tag-tick').text
            chevening = 'True'
            uni = uni.find_all('a')
            uni = uni[1].text

        except:
            chevening = 'False'
            uni = uni.find('a').text 

        #Try to find unis with more than one course

        course_list = section.find_all('li')
        for course in course_list:
            link = course.find('a', class_= 'course-name')
            link = link.get('href') 
            link = 'https://www.postgrad.com' + link
            links.append(link)
            course  = course.find('a', class_= 'course-name').text
            courses.append(course)
            unis.append(uni)
            chevenings.append(chevening)
            fields.append(field)
        #courses.append(course.find('a', class_= 'course-name').text)

    df = pd.DataFrame(list(zip(unis, courses,chevenings,links,fields)),
                columns =['University', 'Courses','Chevening Partner','Links','Field'])
    return df

In [8]:
type(sections_list[0])

tuple

In [9]:
#Using dataFramer function on all sections that and all data to be appended to dataframe
dataframe_list = []
for sections in sections_list:
    section_data = dataFramer(sections)
    dataframe_list.append(section_data)
all_data =pd.concat(dataframe_list)

In [10]:
all_data
all_data.to_csv('chevening_university.csv',index=False)

In [11]:
all_data

,University,Courses,Chevening,Links,Field
0,University of Aberdeen,Transport and Intelligent Mobility,False,https://www.postgrad.com/university-of-aberdee...,transport
1,Cardiff University,Transport and Planning,True,https://www.postgrad.com/cardiff-university-sc...,transport
2,Coventry University,Automotive and Transport Design,True,https://www.postgrad.com/coventry-university-s...,transport
3,Coventry University,Air Transport Management,True,https://www.postgrad.com/coventry-university-s...,transport
4,Coventry University,Highways and Transportation Engineering,True,https://www.postgrad.com/coventry-university-s...,transport
...,...,...,...,...,...
5,University of Lincoln,Cloud Computing,False,https://www.postgrad.com/university-of-lincoln...,cloud
6,Newcastle University,Cloud Computing,True,https://www.postgrad.com/newcastle-university-...,cloud
7,Nottingham Trent University,Cloud and Enterprise Computing,True,https://www.postgrad.com/nottingham-trent-univ...,cloud
8,University of Leicester,Cloud Computing,False,https://www.postgrad.com/university-of-leicest...,cloud
